In [1]:
import requests
import pandas as pd

from dagster import (
    asset, 
    get_dagster_logger,
    AssetExecutionContext,
    MetadataValue,
)



In [2]:
url = "https://realtor.p.rapidapi.com/properties/v3/list"

payload = {
    "limit": 200,
    "offset": 0,
    "postal_code": "90004",
    "status": ["for_sale", "ready_to_build"],
    "sort": {
        "direction": "desc",
        "field": "list_date"
    }
}
headers = {
    "content-type": "application/json",
    "X-RapidAPI-Key": "c330c4c649mshc1af4c3b4deec68p1afe62jsn4d6018d32514",
    "X-RapidAPI-Host": "realtor.p.rapidapi.com"
}

response = requests.post(url, json=payload, headers=headers)

json_data = response.json()


In [10]:
json_data

{'data': {'home_search': {'__typename': 'SearchHomeResult',
   'count': 86,
   'total': 86,
   'results': [{'__typename': 'SearchHome',
     'property_id': '2777398797',
     'listing_id': '2960245999',
     'plan_id': None,
     'status': 'for_sale',
     'photo_count': 40,
     'branding': [{'__typename': 'Branding',
       'photo': None,
       'name': 'Coldwell Banker Realty',
       'phone': None,
       'link': None}],
     'location': {'__typename': 'SearchHomeLocation',
      'address': {'__typename': 'SearchHomeAddress',
       'city': 'Los Angeles',
       'line': '316 N Rossmore Ave Apt 603',
       'street_name': 'Rossmore',
       'street_number': '316',
       'street_suffix': 'Ave',
       'country': 'USA',
       'postal_code': '90004',
       'state_code': 'CA',
       'state': 'California',
       'coordinate': {'__typename': 'Coordinate',
        'lat': 34.077355,
        'lon': -118.326857,
        'accuracy': None}},
      'street_view_url': '',
      'county': {'_

In [29]:
df = pd.DataFrame.from_dict(json_data['data']['home_search']['results'])
df.head()

,__typename,property_id,listing_id,plan_id,status,photo_count,branding,location,open_houses,description,...,list_price,list_price_min,list_price_max,price_reduced_amount,estimate,lead_attributes,last_sold_date,list_date,products,last_sold_price
0,SearchHome,2777398797,2960245999,None,for_sale,40,"[{'__typename': 'Branding', 'photo': None, 'na...","{'__typename': 'SearchHomeLocation', 'address'...","[{'__typename': 'HomeOpenHouse', 'start_date':...","{'__typename': 'SearchHomeDescription', 'sub_t...",...,1100000,None,None,NaN,None,"{'__typename': 'LeadAttributes', 'lead_type': ...",2016-05-17,2023-10-04T18:10:00Z,"{'__typename': 'ProductSummary', 'brand_name':...",535000.0
1,SearchHome,9182943569,2960187572,None,for_sale,13,"[{'__typename': 'Branding', 'photo': None, 'na...","{'__typename': 'SearchHomeLocation', 'address'...",None,"{'__typename': 'SearchHomeDescription', 'sub_t...",...,2600000,None,None,NaN,"{'__typename': 'LatestEstimate', 'estimate': 5...","{'__typename': 'LeadAttributes', 'lead_type': ...",None,2023-10-02T23:39:48Z,"{'__typename': 'ProductSummary', 'brand_name':...",NaN
2,SearchHome,9254094221,2960181923,None,for_sale,1,"[{'__typename': 'Branding', 'photo': None, 'na...","{'__typename': 'SearchHomeLocation', 'address'...",None,"{'__typename': 'SearchHomeDescription', 'sub_t...",...,3990000,None,None,NaN,None,"{'__typename': 'LeadAttributes', 'lead_type': ...",None,2023-10-02T20:39:08Z,"{'__typename': 'ProductSummary', 'brand_name':...",NaN
3,SearchHome,1630794345,2960144176,None,for_sale,50,"[{'__typename': 'Branding', 'photo': None, 'na...","{'__typename': 'SearchHomeLocation', 'address'...","[{'__typename': 'HomeOpenHouse', 'start_date':...","{'__typename': 'SearchHomeDescription', 'sub_t...",...,1890000,None,None,NaN,None,"{'__typename': 'LeadAttributes', 'lead_type': ...",2005-06-01,2023-09-30T19:05:28Z,"{'__typename': 'ProductSummary', 'brand_name':...",900000.0
4,SearchHome,9702145550,2960142135,None,for_sale,13,"[{'__typename': 'Branding', 'photo': None, 'na...","{'__typename': 'SearchHomeLocation', 'address'...","[{'__typename': 'HomeOpenHouse', 'start_date':...","{'__typename': 'SearchHomeDescription', 'sub_t...",...,599000,None,None,NaN,None,"{'__typename': 'LeadAttributes', 'lead_type': ...",None,2023-09-30T16:47:48Z,"{'__typename': 'ProductSummary', 'brand_name':...",NaN


In [43]:
df.columns

Index(['__typename', 'property_id', 'listing_id', 'plan_id', 'status',
       'photo_count', 'branding', 'location', 'open_houses', 'description',
       'virtual_tours', 'matterport', 'advertisers', 'flags', 'source',
       'pet_policy', 'community', 'primary_photo', 'href', 'list_price',
       'list_price_min', 'list_price_max', 'price_reduced_amount', 'estimate',
       'lead_attributes', 'last_sold_date', 'list_date', 'products',
       'last_sold_price'],
      dtype='object')

In [44]:
df['virtual_tours'][0]

[{'__typename': 'VirtualTour',
  'href': 'https://lovelandcarr.com/featured-listings/'}]

In [ ]:
WITH UnnestedBranding AS (
    SELECT
        property_id,
        listing_id,
        b['photo'] AS photo,
        b['name'] AS name,
        b['phone'] AS phone,
        b['link'] AS link,
    FROM raw.call_property_list_api,
    LATERAL UNNEST(branding) AS t(b)
),

UnnestedOpenHouses AS (
    SELECT
        property_id,
        listing_id,
        oh['start_date'] AS open_house_start_time,
        oh['description'] AS open_house_description,
        oh['time_zone'] AS open_house_time_zone,
    FROM raw.call_property_list_api,
    LATERAL UNNEST(open_houses) AS t(oh)
),

VirtualTours AS (
    SELECT
        property_id,
        listing_id,
        vt['href'] as virtual_tour_link
    FROM raw.call_property_list_api,
    LATERAL UNNEST(virtual_tours) as t(vt)
),

Advertisers AS (
    SELECT
        property_id,
        listing_id,
        ad.fulfillment_id as advertiser_fulfillment_id,
        ad.name as advertiser_name,
        ad.email as advertiser_email,
        ad.href as advertiser_href,
        ad.slogan as advertiser_slogan,
        ad.type as advertiser_type
    FROM raw.call_property_list_api,
    LATERAL UNNEST(advertisers) as t(ad)
),

Source AS (
    SELECT
        property_id,
        listing_id,
        ag.agent_id as source_agent_id,
        ag.agent_name as source_agent_name,
        ag.office_id as source_office_id,
        ag.office_name as source_office_name,
        source['id'] as source_id,
        source['type'] as source_type,
        source['spec_id'] as source_spec_id,
        source['plan_id'] as source_plan_id,
        source['listing_href'] as source_listing_href,
        source['listing_id'] as source_listing_id
    FROM raw.call_property_list_api,
    LATERAL UNNEST(source['agents']) as t(ag)
)


SELECT
    a.property_id,
    a.listing_id,
    a.plan_id,
    a.status,
    a.photo_count,
    ub.photo,
    ub.name,
    ub.phone,
    ub.link,
    a.location['address']['city'] AS city,
    a.location['address']['line'] AS line,
    a.location['address']['street_name'] AS street_name,
    a.location['address']['street_number'] AS street_number,
    a.location['address']['street_suffix'] AS street_suffix,
    a.location['address']['country'] AS country,
    a.location['address']['postal_code'] AS postal_code,
    a.location['address']['state_code'] AS state_code,
    a.location['address']['state'] AS state,
    a.location['address']['coordinate']['lat'] AS lat,
    a.location['address']['coordinate']['lon'] AS lon,
    a.location['address']['coordinate']['accuracy'] AS accuracy,
    a.location['street_view_url'] AS street_view_url,
    a.location['county']['fips_code'] AS fips_code,
    uoh.open_house_start_time,
    uoh.open_house_description,
    uoh.open_house_time_zone,
    a.description['sub_type'] AS sub_type,
    a.description['type'] AS type,
    a.description['beds'] AS beds,
    a.description['baths'] AS baths,
    a.description['lot_sqft'] AS lot_sqft,
    a.description['beds_max'] AS beds_max,
    a.description['beds_min'] AS beds_min,
    a.description['sqft_max'] AS sqft_max,
    a.description['sqft_min'] AS sqft_min,
    a.description['baths_full'] AS baths_full,
    a.description['baths_half'] AS baths_half,
    a.description['baths_max'] AS baths_max,
    a.description['baths_full_calc'] AS baths_full_calc,
    a.description['baths_partial_calc'] AS baths_partial_calc,
    vt.virtual_tour_link,
    a.matterport,
    ad.advertiser_fulfillment_id,
    ad.advertiser_name,
    ad.advertiser_email,
    ad.advertiser_href,
    ad.advertiser_slogan,
    ad.advertiser_type,
    a.flags['is_price_reduced'],
    a.flags['is_new_construction'],
    a.flags['is_foreclosure'],
    a.flags['is_plan'],
    a.flags['is_new_listing'],
    a.flags['is_coming_soon'],
    a.flags['is_contingent'],
    a.flags['is_pending'],
    source_agent_id,
    source_agent_name,
    source_office_id,
    source_office_name,
    source_id,
    source_type,
    source_spec_id,
    source_plan_id,
    source_listing_href,
    source_listing_id,
    a.pet_policy['cats'] as cats_allowed,
    a.pet_policy['dogs'] as dogs_allowed,
    a.community,
    a.primary_photo['href'] as primary_photo_href,
    a.href as listing_href,
    a.list_price,
    a.list_price_min,
    a.list_price_max,
    a.price_reduced_amount,
    a.estimate,
    a.lead_attributes['lead_type'] as lead_type,
    a.lead_attributes['show_contact_an_agent'] as show_contact_an_agent,
    a.lead_attributes['opcity_lead_attributes']['flip_the_market_enabled'] as flip_the_market_enabled,
    a.last_sold_date,
    a.list_date,
    a.products['brand_name'] as product_brand_name,
    a.last_sold_price
FROM raw.call_property_list_api a
LEFT JOIN UnnestedOpenHouses uoh ON 
    uoh.property_id = a.property_id AND uoh.listing_id = a.listing_id
LEFT JOIN UnnestedBranding ub ON
    ub.property_id = a.property_id AND ub.listing_id = a.listing_id
LEFT JOIN VirtualTours vt ON 
    vt.property_id = a.property_id AND vt.listing_id = a.listing_id
LEFT JOIN Advertisers ad ON 
    ad.property_id = a.property_id AND ad.listing_id = a.listing_id
LEFT JOIN Source sc ON 
    sc.property_id = a.property_id AND sc.listing_id = a.listing_id;

In [ ]:
Source AS (
    SELECT
        property_id,
        listing_id,
        ag.agent_id as source_agent_id,
        ag.agent_name as source_agent_name,
        ag.office_id as source_office_id,
        ag.office_name as source_office_name,
        source['id'] as source_id,
        source['type'] as source_id,
        source['spec_id'] as source_id,
        source['plan_id'] as source_id,
        source['listing_href'] as source_id,
        source['listing_id'] as source_id
    FROM raw.call_property_list_api,
    LATERAL UNNEST(source['agents']) as t(ag)
)
